# Finding the global optimum of *f(x, y)* and *g(x, y)* with PSO

Finding the global optimum of a function, especially in complex and high-dimensional spaces, is a fundamental challenge in optimization. This task involves searching for a solution that yields the best (maximum or minimum) value of a given objective function, considering that there may be multiple local optima that are not the best possible solution. The presence of multiple local optima makes it difficult for many optimization algorithms, especially deterministic ones, to guarantee finding the global optimum.

The main challenge in finding the global optimum lies in efficiently exploring the search space without getting trapped in local optima. This requires a balance between exploration (searching new areas of the space) and exploitation (refining solutions in promising areas). The complexity of the problem increases with the dimensionality of the search space and the landscape of the objective function, including its ruggedness and the presence of constraints.

### Particle Swarm Optimization (PSO)

Particle Swarm Optimization (PSO) is a metaheuristic algorithm inspired by the social behavior of birds flocking or fish schooling. It is particularly used for finding the global optimum in complex search spaces. Initially, particles are dispersed widely to explore the search space. Over time, as particles update their velocities and positions, they tend to converge towards the area containing the global best solution, balancing exploration and exploitation. Also Unlike gradient-based optimization methods, PSO does not require gradient information of the objective function, making it suitable for optimizing non-differentiable or noisy functions. The algorithm is as follows:

1. **Population-Based**: PSO maintains a population (swarm) of candidate solutions (particles). Each particle represents a potential solution to the optimization problem. In our case, each particle is a tuple `(x, y)` for our 2D functions.

2. **Particle Movement**: Every particle has a velocity that guides its movement in the search space. The movement is influenced by the particle's personal best position and the global best position found by the swarm.

3. **Social and Cognitive Components**: Each particle updates its velocity and position based on its own experience (cognitive component) and the experience of neighboring particles or the entire swarm (social component). This allows particles to move towards better regions of the search space.

PSO's effectiveness depends on several factors, such as the choice of parameters (e.g., size of the swarm, coefficients affecting the social and cognitive components), the nature of the objective function, and the complexity of the problem space. Tuning these parameters appropriately can significantly enhance the algorithm's performance in different optimization tasks.

In [273]:
#importing the necessary libraries
import random
import matplotlib.pyplot as plt
import math

In [ ]:
random.seed(0)

The algorithm will be evaluated using the following functions:

$$f(x, y) = \left| \sin(x) \cdot \cos(y) \cdot \exp\left(\left| 1 - \frac{\sqrt{x^2 + y^2}}{\pi} \right|\right) \right|$$

$$g(x, y) = \frac{x \cdot \sin\left(\pi \cdot \cos(x) \cdot \tan(y)\right) \cdot \sin\left(\frac{y}{x}\right)}{1 + \cos\left(\frac{y}{x}\right)}$$


In [274]:
# The two sample functions
def f(x, y):
    return abs(math.sin(x) * math.cos(y) * math.exp(abs(1 - (math.sqrt(x ` 2 + y ` 2) / math.pi))))     

def g(x, y):
    return (x * math.sin(math.pi * math.cos(x) * math.tan(y)) * math.sin(y/x)) / (1 + math.cos(y/x))

Each particle is defined by a class that encapsulates several key attributes essential for navigating the search space and evaluating performance within the PSO framework. These attributes include:

1. `function`: This attribute specifies the predefined function name that the particle will use to compute its cost. It can either be 'f' or 'g', dictating the cost function to be applied. This choice determines how the particle evaluates its position in the search space.

2. `position`: Represents the particle's current coordinates (x, y) in the search space. The initial position is randomly generated within the defined minimum and maximum bounds for each variable, allowing for an exploration of the solution space right from the start.

3. `velocity`: This is a vector indicating the rate and direction at which the particle moves across the search space. Velocity adjustments are crucial for guiding the particle towards promising regions based on its own experience and the swarm's global knowledge.

4. `best_position`: This records the particle's optimal position encountered so far, based on the cost function's evaluation. It serves as a historical benchmark, influencing the particle's future movements through its cognitive component, allowing it to leverage personal insights for better decision-making.

Furthermore, each particle possesses can calculate its solution cost, which involves evaluating the predefined functions 'f(x, y)' or 'g(x, y)'. The selection between 'f' and 'g' depends on the `function` attribute:

- For function 'g', special attention is needed to avoid division by zero errors in the term $1+\cos(\frac{y}{x}$. If the denominator $x$ in $y/x$ becomes zero, the function is designed to return a cost value of `inf`. This ensures that any position leading to such an undefined state is effectively excluded from being considered as a viable solution, as assigning it an infinite cost renders it an unsuitable candidate for achieving the minimum value of ‘g(x, y)’.

In [1]:
class Particle:
    def __init__(self, func, min_var, max_var):
        self.function = func
        self.position = [random.uniform(min_var, max_var), random.uniform(min_var, max_var)]
        self.velocity = [0, 0]
        self.best_position = self.position

    def cost(self):
        x, y = self.position
        if self.function == 'f':
            return f(x, y)
        else:
            if math.cos(y/x) == -1:
                return float('inf')
            return g(x, y)

The PSO_f and PSO_g functions, while essentially the same in structure, are differet in their variable interval settings for minimization/maximization and their fitness evaluation criteria, with 'f' aiming for higher fitness values and 'g' targeting lower values as more desirable. The following attributes are adjustable for optimal performance:

- `n_particles`: This determines the swarm size, i.e., the total number of particles.
- `n_it`: This specifies the total number of iterations to be conducted.
- `w`: This is the inertial weight, influencing the momentum of particles.
- `c1`: Known as the cognitive component, it adjusts the particle's position based on the particle's own best found position.
- `c2`: Termed as the social component, it modifies the particle's trajectory towards the globally recognized optimal position within the swarm.

The process begins by initializing a swarm consisting of 'n_particles' with randomly assigned positions. Subsequently, for a predetermined number of iterations ('n_it'), each particle's global and personal best positions are updated according to its fitness value. The velocity and position of each particle are recalculated according to the follwoing equations:

$$X(t+1) = X(t) + V(t+1)$$

$$V(t+1) = wV(t) + c_1*rand()*(X_{pbest} - X(t)) + c_2*rand()*(X_{gbest} - X(t))$$


In [276]:
def PSO_f(n_particles=60, n_it=10000, w=0.5, c1=1.0, c2=2.0):
    swarm = [Particle('f', -10, 10) for _ in range(n_particles)]
    global_best_position = None
    global_best_fitness = float('-inf')  # Initialize with negative infinity for f(x, y)

    for _ in range(n_it):
        for particle in swarm:
            fitness = particle.cost()
            if fitness > global_best_fitness:  # Update if fitness is greater
                global_best_fitness = fitness
                global_best_position = particle.position

            if fitness > particle.cost():  # Update personal best if fitness is greater
                particle.best_position = particle.position

            for i in range(2):
                r1 = random.uniform(0, 1)
                r2 = random.uniform(0, 1)
                
                particle.velocity[i] = (w * particle.velocity[i] +
                                        c1 * r1 * (particle.best_position[i] - particle.position[i]) +
                                        c2 * r2 * (global_best_position[i] - particle.position[i]))
                particle.position[i] += particle.velocity[i]

                # Ensure the particle stays within the search space
                particle.position[i] = max(min(particle.position[i], 10), -10)

    return global_best_position, global_best_fitness


In [277]:
def PSO_g(n_particles=60, n_it=10000, w=0.5, c1=1.0, c2=2.0):
    swarm = [Particle('g', -100, 100) for _ in range(n_particles)]
    global_best_position = None
    global_best_fitness = float('inf')  # Initialize with infinity for g(x, y)

    for _ in range(n_it):
        for particle in swarm:
            fitness = particle.cost()
            if fitness < global_best_fitness:  # Update if fitness is lower
                global_best_fitness = fitness
                global_best_position = particle.position

            if fitness < particle.cost():  # Update personal best if fitness is lower
                particle.best_position = particle.position

            for i in range(2):
                r1 = random.uniform(0, 1)
                r2 = random.uniform(0, 1)
                
                particle.velocity[i] = (w * particle.velocity[i] +
                                        c1 * r1 * (particle.best_position[i] - particle.position[i]) +
                                        c2 * r2 * (global_best_position[i] - particle.position[i]))
                particle.position[i] += particle.velocity[i]

                # Ensure the particle stays within the search space
                particle.position[i] = max(min(particle.position[i], 100), -100)

    return global_best_position, global_best_fitness


In [400]:
best_position, best_fitness = PSO_g(n_particles=90, n_it=1000, w=0.1, c1=4.0, c2=2.0)
print("Optimal Solution:")
print("    x =", best_position[0])
print("    y =", best_position[1])
print("Minimum Value:", best_fitness)

Optimal Solution:
    x = -24.445836964440378
    y = 76.79886226447924
Minimum Value: -4018466616.530831


In [381]:
best_position, best_fitness = PSO_f(n_particles=60, n_it=1000, w=0.1, c1=4.0, c2=1.0)
print("Optimal Solution:")
print("    x =", best_position[0])
print("    y =", best_position[1])
print("Maximum Value:", best_fitness)

Optimal Solution:
    x = -8.054804413029489
    y = -9.664923307110666
Maximum Value: 19.20850210820464
